In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
import os

sys.path.append("/content/drive/MyDrive/transformer")
# Change working directory to the script's location
os.chdir("/content/drive/MyDrive/transformer")

# raw data
# path_do_data = "data/english.txt"
path_do_data = "data/80652__REPORT__A5-2001-0263__EN.txt"

data_raw = open(path_do_data, encoding="utf-8").read()

In [3]:
!pip install -r /content/drive/MyDrive/transformer/requirements.txt


In [4]:
!python /content/drive/MyDrive/transformer/train_model.py

tokenizer_config.json: 100% 48.0/48.0 [00:00<00:00, 381kB/s]
config.json: 100% 570/570 [00:00<00:00, 5.63MB/s]
vocab.txt: 100% 232k/232k [00:00<00:00, 5.12MB/s]
tokenizer.json: 100% 466k/466k [00:00<00:00, 15.6MB/s]
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1239778 > 512). Running this sequence through the model will result in indexing errors
Model with 89.48M parameters
Training Progress:   0% 0/5000 [00:00<?, ?it/s]step          0 | train loss 10.7007 | val loss 10.6863
Training Progress:  10% 499/5000 [00:37<05:22, 13.97it/s]step        500 | t

In [5]:
# load the model
import json
import torch
from transformers import AutoTokenizer
from model import Transformer
# from train_model import vocab_size
from utils import (
    BATCH_SIZE,
    BLOCK_SIZE,
    DEVICE,
    DROPOUT,
    LEARNING_RATE,
    NUM_EMBED,
    NUM_HEAD,
    NUM_LAYER,
    MAX_ITER,
    EVAL_INTER,
    load_model_from_checkpoint,
    decode,
)

In [18]:
# set parameter

# device path_to_checkpoint(need check and change every time!!) vocab_path
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
path_to_checkpoint = "/content/drive/MyDrive/transformer/checkpoints/checkpoint_epoch-4999_28.10.2024_15:07:16.pt"
vocab_path = "bert-base-uncased"

# find vocab_size
with open("/content/drive/MyDrive/transformer/checkpoints/vocab_info.json", "r") as f:
    vocab_info = json.load(f)
vocab_size = vocab_info["vocab_size"]

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(vocab_path)
vocab_size = tokenizer.vocab_size

# parameter
model = Transformer(
    vocab_size=vocab_size,
    num_embed=NUM_EMBED,
    block_size=BLOCK_SIZE,
    num_heads=NUM_HEAD,
    num_layers=NUM_LAYER,
    dropout=DROPOUT,
).to(DEVICE)

# load_model_from_checkpoint
model = load_model_from_checkpoint(
    model_class=Transformer,
    path_to_checkpoint=path_to_checkpoint,
    vocab_size=vocab_size,
    num_embed=NUM_EMBED,
    block_size=BLOCK_SIZE,
    num_heads=NUM_HEAD,
    num_layers=NUM_LAYER,
    dropout=DROPOUT
)

Successfully loaded model from the checkpoint


In [19]:
model.to(DEVICE)

Transformer(
  (token_embedding_table): Embedding(30522, 768)
  (position_embedding_table): Embedding(64, 768)
  (blocks): Sequential(
    (0): TransformerBlock(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x AttentionHead(
            (key): Linear(in_features=768, out_features=128, bias=False)
            (query): Linear(in_features=768, out_features=128, bias=False)
            (value): Linear(in_features=768, out_features=128, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): ReLU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((768,), eps=1e-05

In [27]:
# # check load or not
# # skip this,next also run sucessfully??

# if model is not None:
#     model.to(DEVICE)

#     # parameters
#     for param in model.parameters():
#         assert param.device == torch.device(DEVICE), "Model parameter is not on the correct device"

#     # generate
#     context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
#     generated_tokens = model.generate(idx=context, max_new_tokens=100, block_size=BLOCK_SIZE)[0]
#     generated_text = decode(enc_sec=generated_tokens, tokenizer=AutoTokenizer.from_pretrained(vocab_path))

#     print("Can Generated text:\n", generated_text)
# else:
#     print("Failed to load the model from the checkpoint.")

AssertionError: Model parameter is not on the correct device

In [28]:
# context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
context = torch.zeros((1, 1), dtype=torch.long).to(DEVICE)
generated_tokens = model.generate(idx=context, max_new_tokens=100, block_size=BLOCK_SIZE)[0]
generated_text = decode(enc_sec=generated_tokens, tokenizer=tokenizer)

print("Generated text:\n", generated_text)

Generated text:
 [PAD] on my favorite authors? do you think i would find work for me? do you think i would find work for me? do you think i'll give it another try? do you think it's a good idea? do you think we need old enough? do you think you have a good appetite? do you think you have a good friend? do you think you have a good day? do you usually have a good day? do you want to listen to me? do you want
